In [1]:
import numpy as np
import pandas as pd
pd.options.display.precision = 2
pd.options.display.max_columns = 50

In [2]:
import re
import os
os.chdir('../../')

In [3]:
from scipy.stats import pearsonr

In [4]:
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
matplotlib.rcParams['figure.figsize'] = [15, 10]

In [5]:
#import seaborn as sns
import plotly
import plotly.graph_objs as go
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
init_notebook_mode(connected=True)

# Read Predictors data

In [6]:
predictors_df = pd.read_csv('data/imf_cpi_indices_yearly.csv', index_col='year')

In [7]:
original_predictors = predictors_df.columns
cols = original_predictors
predictors_df = predictors_df.join(predictors_df[cols].shift(1), rsuffix='_1y_shift')
predictors_df = predictors_df.join(predictors_df[cols].shift(2), rsuffix='_2y_shift')
predictors_df = predictors_df.loc[2003:2019]

In [8]:
predictors_df

,All index,APSP crude oil($/bbl),Coal index,Energy index,Fertilizer,Food and beverage index,Industrial Materials index,Natural gas index,Non-Fuel index,Propane,All index_1y_shift,APSP crude oil($/bbl)_1y_shift,Coal index_1y_shift,Energy index_1y_shift,Fertilizer_1y_shift,Food and beverage index_1y_shift,Industrial Materials index_1y_shift,Natural gas index_1y_shift,Non-Fuel index_1y_shift,Propane_1y_shift,All index_2y_shift,APSP crude oil($/bbl)_2y_shift,Coal index_2y_shift,Energy index_2y_shift,Fertilizer_2y_shift,Food and beverage index_2y_shift,Industrial Materials index_2y_shift,Natural gas index_2y_shift,Non-Fuel index_2y_shift,Propane_2y_shift
year,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2003,65.99,66.92,43.29,78.15,63.89,67.88,56.87,122.67,57.58,118.85,56.14,62.37,39.49,62.99,49.79,62.27,48.89,85.20,51.41,84.21,56.89,61.95,49.34,66.68,54.60,60.23,48.65,103.53,50.12,97.21
2004,79.37,82.50,82.92,98.81,74.88,73.98,72.09,134.16,65.94,152.77,65.99,66.92,43.29,78.15,63.89,67.88,56.87,122.67,57.58,118.85,56.14,62.37,39.49,62.99,49.79,62.27,48.89,85.20,51.41,84.21
2005,97.94,117.35,71.92,136.64,92.23,75.95,83.98,195.76,71.19,187.74,79.37,82.50,82.92,98.81,74.88,73.98,72.09,134.16,65.94,152.77,65.99,66.92,43.29,78.15,63.89,67.88,56.87,122.67,57.58,118.85
2006,114.36,141.65,76.88,154.33,89.30,81.92,121.26,189.10,86.73,208.49,97.94,117.35,71.92,136.64,92.23,75.95,83.98,195.76,71.19,187.74,79.37,82.50,82.92,98.81,74.88,73.98,72.09,134.16,65.94,152.77
2007,129.29,150.35,98.37,169.42,129.20,93.37,144.61,193.41,101.54,248.43,114.36,141.65,76.88,154.33,89.30,81.92,121.26,189.10,86.73,208.49,97.94,117.35,71.92,136.64,92.23,75.95,83.98,195.76,71.19,187.74
2008,163.33,197.57,192.19,234.79,250.28,109.91,134.61,271.00,113.92,290.15,129.29,150.35,98.37,169.42,129.20,93.37,144.61,193.41,101.54,248.43,114.36,141.65,76.88,154.33,89.30,81.92,121.26,189.10,86.73,208.49
2009,117.02,129.47,104.45,144.33,159.09,93.21,114.23,154.14,98.14,173.24,163.33,197.57,192.19,234.79,250.28,109.91,134.61,271.00,113.92,290.15,129.29,150.35,98.37,169.42,129.20,93.37,144.61,193.41,101.54,248.43
2010,146.26,168.02,145.77,179.76,143.15,105.89,169.83,163.65,123.10,239.93,117.02,129.47,104.45,144.33,159.09,93.21,114.23,154.14,98.14,173.24,163.33,197.57,192.19,234.79,250.28,109.91,134.61,271.00,113.92,290.15
2011,182.70,213.81,182.64,233.34,189.55,126.21,195.61,204.33,147.69,301.22,146.26,168.02,145.77,179.76,143.15,105.89,169.83,163.65,123.10,239.93,117.02,129.47,104.45,144.33,159.09,93.21,114.23,154.14,98.14,173.24


# Sponsor Report Dataset

## Read Dataset

In [9]:
types = {
    'Student Id': str,
    'Availability year': str,
    'ETS availability year': str
}
data_df = pd.read_csv('data/sponsored_payments_raw.csv', dtype=types)
data_df.head()

,Student Id,unique,unique 2,Availability year,2019 ETS 7 March 2019,ETS availability year,Email,Family Name,First Given Name,No,Spk Cd,Ver,Att,Internal Org Unit Name,Full Title,Sponsor,Sponsor Name,SPO lookup,SPO country,Account,Finance Category Type,Start Date,End Date,Percentage,Amount,Actual Amount,Status,Notes
0,999888,999888,999888,*na,*na,*na,*na,Li,Qing,1,NaN,0,0,NaN,NaN,SXP-MIR,Melbourne Internat.Research Sch,UoM Scholarship/Bursary/Discount/Ledger,other,4360,Tuition - Overseas,1/1/18,31/12/21,100,0.0,0.0,Proposed,NaN
1,992098,992098,992098,*na,*na,*na,*na,Bai,Xuan,1,NaN,0,0,NaN,NaN,SXP-MIR,Melbourne Internat.Research Sch,UoM Scholarship/Bursary/Discount/Ledger,other,4360,Tuition - Overseas,1/1/18,31/12/21,100,0.0,0.0,Proposed,NaN
2,991642,991642,991642,*na,*na,*na,*na,Zhang,Xiaoyu,1,NaN,0,0,NaN,NaN,SXP-MIR,Melbourne Internat.Research Sch,UoM Scholarship/Bursary/Discount/Ledger,other,4360,Tuition - Overseas,1/1/18,31/12/21,100,0.0,0.0,Proposed,NaN
3,991167,991167,991167,*na,*na,*na,*na,Gao,Zhan,1,NaN,0,0,NaN,NaN,SXP-MIR,Melbourne Internat.Research Sch,UoM Scholarship/Bursary/Discount/Ledger,other,4360,Tuition - Overseas,1/1/18,1/7/21,100,0.0,0.0,Proposed,NaN
4,990308,990308,990308,*na,*na,*na,*na,Tong,Xin,1,NaN,0,0,NaN,NaN,SXP-MIR,Melbourne Internat.Research Sch,UoM Scholarship/Bursary/Discount/Ledger,other,4360,Tuition - Overseas,1/1/18,31/12/21,100,0.0,0.0,Proposed,NaN


## Clean up the data

### Remove entries with Status = 'Proposed'

In [10]:
data_df.Status.value_counts()

Confirmed    36778
Proposed      1831
Name: Status, dtype: int64

In [11]:
mask = data_df.Status == 'Proposed'
data_df[mask].describe().astype(int)

,No,Ver,Att,Account,Percentage,Amount,Actual Amount
count,1831,1831,1831,1831,1831,1831,1831
mean,2,0,0,4360,84,1075,0
std,10,0,0,732,33,4407,0
min,1,0,0,2343,0,0,0
25%,1,0,0,4360,100,0,0
50%,1,0,0,4360,100,0,0
75%,1,0,0,4360,100,0,0
max,182,6,3,12330,100,57600,0


In [12]:
data_df = data_df[~mask]

### Remove entries with "Availability Year" = *na

In [13]:
year_column = 'Availability year'
na_value = '*na'
mask2 = data_df[year_column] == na_value
data_df[mask2].describe().astype(int)

,No,Ver,Att,Account,Percentage,Amount,Actual Amount
count,891,891,891,891,891,891,891
mean,7,1,0,5245,84,580,15099
std,19,1,0,3186,32,2870,26847
min,1,0,0,1,0,0,0
25%,1,1,1,3048,100,0,0
50%,2,1,1,4360,100,0,1196
75%,3,3,1,6800,100,0,17803
max,181,6,4,12551,100,26840,146802


In [14]:
data_df[mask2].describe(include=['object']).T

,count,unique,top,freq
Student Id,891,616,372732,9
unique,891,641,375358751AA,9
unique 2,891,639,375358Ph.D.- Science,9
Availability year,891,1,*na,891
2019 ETS 7 March 2019,891,1,*na,891
ETS availability year,891,1,*na,891
Email,891,1,*na,891
Family Name,891,542,Zhang,10
First Given Name,889,572,Jennifer,10
Spk Cd,733,138,SA-RES,68


In [15]:
data_df = data_df[~mask2]
data_df[year_column] = data_df[year_column].astype(int)

## Dataset summary

In [16]:
data_df.describe(include=['object']).T

,count,unique,top,freq
Student Id,35887,16734,346612,19
unique,35887,18166,600240101AA,13
unique 2,35887,18136,376295Bachelor of Arts,13
2019 ETS 7 March 2019,35887,2,*na,29729
ETS availability year,35887,12,*na,29729
Email,35886,3279,*na,29729
Family Name,35886,10291,Nguyen,389
First Given Name,35527,10421,Muhammad,243
Spk Cd,35631,748,B-COM,3810
Internal Org Unit Name,34994,60,Business and Economics,7584


In [17]:
#data_df['Availability year'].value_counts()#.head(10)

In [18]:
data_df.describe().astype(int).T

,count,mean,std,min,25%,50%,75%,max
Availability year,35887,2012,3,2003,2010,2013,2015,2019
No,35887,11,23,1,1,2,8,319
Ver,35887,2,1,0,1,2,3,7
Att,35887,1,0,0,1,1,1,6
Account,35887,4845,2289,1,3089,4360,6800,12552
Percentage,35887,71,42,0,20,100,100,100
Amount,35887,1056,3574,0,0,0,0,78458
Actual Amount,35887,20054,26665,0,2472,12552,27424,189705


## Convert data into response format

In [19]:
# configs for analysis
response_config = {
    'student_count':
    {
        'source_dataset': 
        {
            'column_name': 'Student Id',
            'agg_func': pd.Series.nunique
        },        
        'chart_properties':
        {
            'title': 'Number of Unique Sponsored Students',               
        }
    },
    'total_amount':
    {
        'source_dataset': 
        {
            'column_name': 'Actual Amount',
            'agg_func': sum
        },        
        'chart_properties':
        {
            'title': 'Total Paid Amount',               
        }
    }
}

factors_config = {
    'country': {
        'source_dataset':
        {
            'column_name': 'SPO country',
        },        
        'chart_properties':
        {
            'title': 'Country',               
        }
    },
    'sponsor_body':
    {
        'source_dataset':
        {
            'column_name': 'Sponsor Name',
        },        
        'chart_properties':
        {
            'title': 'Sponsor',
        }
    }
}
time_factor = 'year'
time_factor_config = {
    'title': 'Year'
}
factors_names = list(factors_config.keys())
LEVELS_TOP = 20
CONFIDENCE_LEVEL = 0.05

In [20]:
# prepare dataframe with expected column names
col_names_map = {factors_config[key]['source_dataset']['column_name']: key for key in factors_config}
col_names_map[year_column] = time_factor
source_df = data_df.rename(columns=col_names_map)
#source_df.index.name = time_factor

In [21]:
# aggregate raw data to get response data
temp_dfs = []

group_columns = [time_factor] + factors_names
for key in response_config.keys():
    config = response_config[key]['source_dataset']
    df = source_df.groupby(group_columns)[config['column_name']].agg(config['agg_func'])
    df.name = key
    temp_dfs.append(df)
    
response_df = pd.concat(temp_dfs, axis=1)

# Visualise

In [22]:
def plot_bar_chart(df, y_config, x_config):
    traces = []
    traces.append(go.Bar(
            x = df.index.values,
            y = df.values,
            name = y_config['title'],
            opacity = 0.5))


    layout = dict(
        title = '{} By {}'.format(y_config['title'], x_config['title']),
        xaxis = dict(title = x_config['title']),

        yaxis = dict(
            title = y_config['title'],
            #range = [0, 250]
        ),

        legend=dict(
            x=0.3,
            y=1.1,
            traceorder='normal',
            font=dict(
                family='sans-serif',
                size=12,
                color='#000'
            ),
            orientation="h"
        )
    ,
    )

    fig = dict(data=traces, layout=layout)
    iplot(fig)

In [23]:
def plot_heatmap(correlation_df, response_var, factor_var):
    df = correlation_df.loc[correlation_df.index.values[::-1]]
    trace = go.Heatmap(z=df.values,
                       x=df.columns.values,
                       y=df.index.values,
                       zmin = -1,
                       zmax = 1
                      )
    adjust = lambda s: s.replace('_', ' ').title()
    response_var.capitalize()
    layout = go.Layout(
        title="Correlation Between {} and Commodity Indices by {}".format(adjust(response_var), 
                                                                              adjust(factor_var)),
        autosize=False,
        margin = dict(
                r = 10,
                t = 25,
                b = 200,
                l = 200),
        width=500,
        height=700)

    fig = go.Figure(data=[trace], layout=layout)
    iplot(fig)

In [24]:
def plot_comparison_plot(resp_df, predictors_df, predictors, x_config, y_config):
    traces = []

    # Adding predictor lines
    for predictor in predictors:
        traces.append(
            go.Scatter(
                x=predictors_df.index.values,
                y=predictors_df[predictor].values,
                name=predictor,
                mode='lines'
            ),
        )

    # Adding response variable bar-chart by factor level
    factor_levels = resp_df.columns
    visibility = True
    for level in factor_levels:
        x = resp_df.index.values
        traces.append(go.Bar(
            x=x,
            y=resp_df[level],
            yaxis='y2',
            visible=visibility,
            name=y_config['title'],
            marker=dict(
                color='rgba(50, 171, 96, 0.7)',
                line=dict(
                    color='rgba(50, 171, 96, 1.0)',
                    width=2,
                )),
            opacity = 0.5))
        visibility = False


    # Adding dropdown menu    
    buttons = []
    pr_count = len(predictors)
    for idx, level in enumerate(factor_levels):
        mask = [True] * pr_count + [False] * len(factor_levels)
        mask[pr_count + idx] = True
        buttons.append(
            dict(
                label = level,
                method = 'update',
                args = [{'visible': mask}]
            )
        )


    layout = dict(
        title = 'Trends in Commodity Prices and Number of Sponsored Students by Country',
        xaxis = dict(title = 'Year'),

        yaxis = dict(
            title = 'Commodity Index Value',
            range = [0, 250]
        ),

        yaxis2 = dict(
            title=y_config['title'],
            overlaying='y',
            side='right',
            range=[0, 800]#df.max().max() - 0]
            #position=0.15
        ),
        updatemenus = list([dict(
                active=0,
                pad = {'r': 0, 't': 0},
                x = 0.0,
                xanchor = 'left',
                y = 1.1,
                yanchor = 'top',
                buttons=buttons)]),
        legend=dict(
            x=0.3,
            y=1.1,
            traceorder='normal',
            font=dict(
                family='sans-serif',
                size=12,
                color='#000'
            ),
            orientation="h"
        ),
        margin = dict(
                r = 50,
                t = 100,
                b = 50,
                l = 50)
    ,
    )


    fig = dict(data=traces, layout=layout)

    iplot(fig)

In [25]:
def aggregate_by_factor(response_df, response_var, time_factor, factor_var, levels_top=20):
    group_columns = [time_factor, factor_var]
    resp_by_factor_df = response_df.groupby(level=group_columns)[response_var].agg(sum).unstack(factor_var, fill_value=0)
    top_levels = resp_by_factor_df.sum(axis=0).sort_values(ascending=False).index.values
    top_levels = top_levels[:min(top_levels.shape[0], levels_top)]
    return resp_by_factor_df[top_levels]

## Number of Sponsored Students

In [26]:
response_var = list(response_config.keys())[0]

In [27]:
df = response_df[response_var].groupby(level=time_factor).agg(sum)
y_config = response_config[response_var]['chart_properties']
plot_bar_chart(df, y_config, time_factor_config)

In [28]:
for factor_var in list(factors_config.keys()):
    df = response_df[response_var].groupby(level=factor_var).agg(sum).sort_values(ascending=False).head(15)
    y_config = response_config[response_var]['chart_properties']
    x_config = factors_config[factor_var]['chart_properties']
    plot_bar_chart(df, y_config, x_config)

### Correlation

In [29]:
#mask = correlation_df.columns[-10:]

In [30]:
def plot_heatmap(correlation_df, response_var, factor_var):
    df = correlation_df.loc[correlation_df.index.values[::-1]]
    trace = go.Heatmap(z=df.values,
                       x=df.columns.values,
                       y=df.index.values,
                       zmin = -1,
                       zmax = 1
                      )
    adjust = lambda s: s.replace('_', ' ').title()
    response_var.capitalize()
    layout = go.Layout(
        title="Correlation Between {} <br>and Commodity Indices by {}".format(adjust(response_var), 
                                                                              adjust(factor_var)),
        autosize=False,
        margin = dict(
                r = 10,
                t = 50,
                b = 200,
                l = 200),
        width=850,
        height=700)

    fig = go.Figure(data=[trace], layout=layout)
    iplot(fig)

In [31]:
time_factor_values = predictors_df.index.values

for factor_var in list(factors_config.keys()):
    # transform response df to get data for particular factor
    resp_by_factor_df = aggregate_by_factor(response_df, response_var, time_factor, factor_var)
    resp_by_factor_df = resp_by_factor_df.loc[time_factor_values]

    # Calculate correlation and pvalues
    factor_levels = resp_by_factor_df.columns
    predictors = predictors_df.columns
    correlations = np.zeros((factor_levels.shape[0], predictors.shape[0]))
    pvalues = np.zeros((factor_levels.shape[0], predictors.shape[0]))
    for idx, level in enumerate(factor_levels):
        for jdx, predictor in enumerate(predictors):
            correlation, pvalue = pearsonr(resp_by_factor_df[level], predictors_df[predictor])
            correlations[idx, jdx] = correlation
            pvalues[idx, jdx] = pvalue

    # Form dataframe with correlations
    correlation_df = pd.DataFrame(data=correlations, index=factor_levels, columns=predictors)
    pvalues_df = pd.DataFrame(data=pvalues, index=factor_levels, columns=predictors)
    significance_mask = pvalues_df < CONFIDENCE_LEVEL
    correlation_df[~significance_mask] = np.nan


    #plot_heatmap(correlation_df[mask], response_var, factor_var)
    plot_heatmap(correlation_df, response_var, factor_var)

### Comparison Chart

In [32]:
#predictors = original_predictors[:5]
predictors = ['Non-Fuel index_2y_shift']
for factor_var in list(factors_config.keys()):
    y_config = response_config[response_var]['chart_properties']
    x_config = factors_config[factor_var]['chart_properties']
    df = aggregate_by_factor(response_df, response_var, time_factor, factor_var)
    plot_comparison_plot(df, predictors_df, predictors, x_config, y_config)


## Total Paid Amount

In [33]:
response_var = list(response_config.keys())[1]

In [34]:
df = response_df[response_var].groupby(level=time_factor).agg(sum)
y_config = response_config[response_var]['chart_properties']
plot_bar_chart(df, y_config, time_factor_config)

In [35]:
for factor_var in list(factors_config.keys()):
    df = response_df[response_var].groupby(level=factor_var).agg(sum).sort_values(ascending=False).head(15)
    y_config = response_config[response_var]['chart_properties']
    x_config = factors_config[factor_var]['chart_properties']
    plot_bar_chart(df, y_config, x_config)

### Correlation

In [36]:
time_factor_values = predictors_df.index.values

for factor_var in list(factors_config.keys()):
    # transform response df to get data for particular factor
    resp_by_factor_df = aggregate_by_factor(response_df, response_var, time_factor, factor_var)
    resp_by_factor_df = resp_by_factor_df.loc[time_factor_values]

    # Calculate correlation and pvalues
    factor_levels = resp_by_factor_df.columns
    predictors = predictors_df.columns
    correlations = np.zeros((factor_levels.shape[0], predictors.shape[0]))
    pvalues = np.zeros((factor_levels.shape[0], predictors.shape[0]))
    for idx, level in enumerate(factor_levels):
        for jdx, predictor in enumerate(predictors):
            correlation, pvalue = pearsonr(resp_by_factor_df[level], predictors_df[predictor])
            correlations[idx, jdx] = correlation
            pvalues[idx, jdx] = pvalue

    # Form dataframe with correlations
    correlation_df = pd.DataFrame(data=correlations, index=factor_levels, columns=predictors)
    pvalues_df = pd.DataFrame(data=pvalues, index=factor_levels, columns=predictors)
    significance_mask = pvalues_df < CONFIDENCE_LEVEL
    correlation_df[~significance_mask] = np.nan


    plot_heatmap(correlation_df, response_var, factor_var)

### Comparison Chart

In [37]:
predictors = original_predictors[:5]
for factor_var in list(factors_config.keys()):
    y_config = response_config[response_var]['chart_properties']
    x_config = factors_config[factor_var]['chart_properties']
    df = aggregate_by_factor(response_df, response_var, time_factor, factor_var)
    plot_comparison_plot(df, predictors_df, predictors, x_config, y_config)
